In [ ]:
# In dieser Zelle werden alle benötigten Pakete für die Auswertung importiert. Außerdem werden alle benötigten Konstanten definiert.

# benötigte Pakete
import matplotlib.pyplot as plt
import numpy as np
import sympy as smp
import scipy as scp
from scipy.integrate import quad
import scipy.constants as const
from scipy.optimize import minimize, minimize_scalar, brentq
from scipy.interpolate import interp1d
from scipy.special import zeta, digamma
import pandas as pd
import math

# Rydberg-Konstante
Rydberg_CODATA = const.Rydberg #in m^-1

#Feinstrukturkonstante
alpha=const.alpha

#Bohrradius
a_0=5.29177210544e-11 #aus Codata, Bohrradius in m
a_eV = 1/(alpha*0.511e6) # in eV
a_fm = 52917.7249 # in fm

# Berechnung der Rydberg-Energie
Rydberg_energy = Rydberg_CODATA*const.h*const.c

# Elektronmasse
m_e_kg=const.m_e 
m_e=m_e_kg#in kg
# m_e=m_e_kg*5.60958616721986e+29 #in MeV

# Protonenmasse
m_p_kg=const.m_p 
m_p=m_p_kg #in kg
# m_p=m_p_kg*5.60958616721986e+29 #in MeV

# Gesamtmasse des Wasserstoffatoms
M = m_e+m_p

# Muoenmasse
m_my_kg = 1.883531627e-28 #in kg
m_my = m_my_kg 

# reduzierte Masse von Wasserstoff
m_r=(m_e*m_p)/(m_e + m_p) 

# Kernladungszahl. Diese wurde mit implimentiert um einen besseren 
# Vergleich mit der Parametrisierung aus der Literatur durchführen zu können
z=1

# reduzierte Comptonwellenlänge
comp_wave_red = const.hbar/(m_e*const.c) #reduzierte comptonwellenlänge in m

# Protonradius
r_N_fm = 0.84075 #bin fm
r_N_CODATA = r_N_fm*1e-15 #in m

# Euler-Mascheroni-Konstante
gamma = 0.5772156649015328606065120900824024310421 

print("done")

done


In [ ]:
# Definition aller Hilfsfunktionen, die für die Berechnung der Matrixelemente benötigt werden

#Grobstruktur Energie in eV
# n ist die Hauptquantenzahl
def Enl(n): 
    return - Rydberg_energy/n**2/const.e

# Radialteil der Wellenfunktion in nat. Einheiten
# n ist die Hauptquantenzahl, l die Bahndrehimpulsquantenzahl, r der Abstand zwischen Kern und Elektron
def R_nl_th(r,n,l):
    return ((2/(n*a_eV))**3*scp.special.factorial(np.abs(n-l-1))/(2*n*scp.special.factorial(np.abs(n+l))))**0.5*np.exp(-(r/(n*a_eV)))*(2*r/(n*a_eV))**l*scp.special.assoc_laguerre(2*r/(n*a_eV),np.abs(n-l-1),np.abs(2*l+1))

# Radialteil der Wellenfunktion in atomaren Einheiten
def R_nl_atomic_units(r,n,l):
    return ((2/(n*a_0))**3*scp.special.factorial(np.abs(n-l-1))/(2*n*scp.special.factorial(np.abs(n+l))))**0.5*np.exp(-(r/(n*a_0)))*(2*r/(n*a_0))**l*scp.special.assoc_laguerre(2*r/(n*a_0),np.abs(n-l-1),np.abs(2*l+1))

# Integrand für die Berechnung der Matrixelemente in nat. Einheiten
def integrand_nat_units(r,n,l,m,c):
    return -c/(4*np.pi)*r*np.exp(-m*r)*(R_nl_th(r,n,l)**2)

# Integrand für die Berechnung der Matrixelemente in nat. Einheiten ohne Kopplung
def integrand_nat_units_without_c(r,n,l,m):
    return -1/(4*np.pi)*r*np.exp(-m*r)*(R_nl_th(r,n,l)**2)

# Integrand für die Berechnung der Matrixelemente in atomaren Einheiten
def integrand_atomic_units(r,n,l,m,c):
    return -137.0359991*c/(4*np.pi)*r*np.exp((-2.68172763e-4)*m*r)*(R_nl_atomic_units(r,n,l)**2)

# Integrand für die Berechnung der Matrixelemente in atomaren Einheiten ohne Kopplung
def integrand_atomic_units_without_c(r,n,l,m):
    return -137.0359991/(4*np.pi)*r*np.exp((-2.68172763e-4)*m*r)*(R_nl_atomic_units(n,l,r)**2) # m in eV, r in a_0

# Definition des Kronecker Delta für skalare Größen
def delta (i,j):
    if i == j:
        return 1
    else:
        return 0

# Kurzschreibweise für die Logarithmusfunktion mit Basis e
def ln(x):
    return np.log(x)

print("done")

done


In [ ]:
#Festlegung der Parameter der Interaktion und Naturkonstanten

# Kopplungskonstante
coupling=1e-12

# Definietion der Arrays für
# die Hauptquantenzahl n
n_array_paper= np.array([5,8,12,18,30])
# die Bahndrehimpulsquantenzahl l
l_array_paper = np.array([1,2,3,4])
# und der Bosonenmasse m
mass_array_paper= np.array([1,10,100,1000])

print("done")

# Berechnung der Matrixelemente der neuen Wechselwirkung für Abb. 2.2.1

# Array für die Speicherung der Integrale. Es wird jeweils die n- und l-Quantenzahl
# und die Masse des Teilchens für jedes Integral gespeichert. Die Integrale werden in nat. Einheiten berechnet
integrals_plot_data = np.zeros([len(n_array_paper),len(mass_array_paper),len(l_array_paper),10])

# Iteratives Durchlaufen aller n-, l-Quantenzahlen für jede Masse m
i=0
for n in n_array_paper:
    ii = 0
    integrals_plot_data[i,0,0,0] = int(n)
    # print(n)
    for m in mass_array_paper:
        iii=0
        # print(m)
        integrals_plot_data[i,ii,0,0] = float(m)

        for l in l_array_paper:
            # print(l)

            # numerische Berechnung mit der Funktion quad über alle positive reelle Zahlen. epsabs legt den absoluten Fehler fest. 
            # Die Ausgabe der Funktion sind das Integral (integrals) und der Fehler (err)
            # inetrgand_nat_units wurde am Anfang definiert und ist der Integrand für die Berechnung der Matrixelemente in nat. Einheiten
            integrals, err = quad(integrand_nat_units, 0, smp.oo, args=(n,l,m,coupling),epsabs=1e-50)

            integrals_plot_data[i,ii,iii,0] = int(l)
            # Speichern der relativen Energieverschiebung
            integrals_plot_data[i,ii,iii,1] = integrals/(Enl(n))
            # Speichern des Fehlers auf die Integrale
            integrals_plot_data[i,ii,iii,2] = err
            # integrals_plot_data[i,ii,iii,3] = integrals/(-const.h*const.c*Ry/n**2*const.e)

            iii = iii + 1
    
        ii = ii + 1

    i = i + 1

# print(integrals_plot_data[:,1,:,1])

[[2.04168013e-11 2.04131325e-11 2.04076300e-11 2.04002944e-11]
 [1.85015384e-11 1.84930535e-11 1.84803299e-11 1.84633718e-11]
 [1.53728894e-11 1.53572570e-11 1.53338235e-11 1.53026070e-11]
 [1.09445576e-11 1.09211899e-11 1.08861879e-11 1.08396108e-11]
 [5.99961243e-12 5.97531571e-12 5.93899167e-12 5.89078488e-12]]


In [ ]:
# Berechnung der Integrale mit varriierenden Massen für feste n aus Abb 2.2.2

# Festlegung der festen Kopplung
coupling=1e-12

# Definierung der Arrays für

# die Bosonenmasse
m_array_mint =np.logspace(0,5.1,300)

# die n-Quantenzahlen
n_array_paper_mint = np.array([5,8,12,26])

# und die l-Quantenzahlen.
l_array_paper_mint = np.array([0,2])

# Definieren der Array für die Speicherung der Integrale. Es wird jeweils die n- und l-Quantenzahl und die Masse des Bosons gespeichert.
# Dazu die relative Energieverschiebung und der Fehler auf die Integrale
integrals_plot_data_mint = np.zeros([len(l_array_paper_mint),len(n_array_paper_mint),len(m_array_mint),4])

# Iteratives Durchlaufen aller n-, l-Quantenzahlen für jede Masse m
i=0
for l in l_array_paper_mint:
    ii = 0
    integrals_plot_data_mint[i,0,0,0] = int(l)

    for n in n_array_paper_mint:
        iii=0
        integrals_plot_data_mint[i,ii,0,0] = int(n)

        for m in m_array_mint:
            # Numerische Berechnung mit der Funktion quad über alle positive reelle Zahlen. epsabs legt den absoluten Fehler fest.
            # Die Ausgabe der Funktion sind das Integral (integrals) und der Fehler (err)
            # inetrgand_nat_units wurde am Anfang definiert und ist der Integrand für die Berechnung der Matrixelemente in nat. Einheiten
            integrals, err = quad(integrand_nat_units, 0, smp.oo, args=(n,l,m,coupling),epsabs=1e-800)

            # Berechnung der realtiven Energieverschiebung zur Grobstrukturenergie
            rel_int = integrals/Enl(n)

            # Speichern der Ergebnisse
            integrals_plot_data_mint[i,ii,iii,0] = float(m)
            integrals_plot_data_mint[i,ii,iii,1] = rel_int
            integrals_plot_data_mint[i,ii,iii,2] = err

            iii = iii + 1
    
        ii = ii + 1

    i = i + 1

print("done")

C:\Users\Thor Frere\AppData\Local\Temp\ipykernel_11856\657413376.py:19: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  integrals, err = quad(integrand_nat_units, 0, smp.oo, args=(n,l,m,coupling),epsabs=1e-800)
C:\Users\Thor Frere\AppData\Local\Temp\ipykernel_11856\657413376.py:19: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  integrals, err = quad(integrand_nat_uni

done


In [ ]:
# Berechnung der Integrale mit festgehaltener realtiver ENergieverschieng, aus denen später die jeweilige Kopplunskonstante berechnet werden kann
# Dazu wird int(c, m) umgeschrieben zu c*int(m). Letzteres wird gleich 10^(-12) gesetzt. Daraus folgt c = 10^(-12)/int(m)

# Definierung der Arrays für das SPeichern der Integrale für die Masse m mit der zugehörigen n- und l-Quantenzahl
c_m_data = np.zeros([len(l_array_paper_mint),len(n_array_paper_mint),len(m_array_mint),2])

# Iteratives DUrchlaufen aller n-, l-Quantenzahlen für jede Masse m
for i in range(0,len(l_array_paper_mint)):
    c_m_data[i,0,0,0]=int(l_array_paper_mint[i])

    for ii in range(0,len(n_array_paper_mint)):
        c_m_data[i,ii,0,0]=int(n_array_paper_mint[ii])

        for iii in range(0,len(m_array_mint)):
            # Berechnug der Integrale ohne c-Abhängigkeit mit der Funktion quad über alle positive reelle Zahlen. epsrel legt den relativen Fehler fest.
            # Die Ausgabe der Funktion sind das Integral (integral) und der Fehler (err)
            # inetrgand_nat_units_without_c wurde am Anfang definiert und ist der Integrand für die Berechnung der Matrixelemente in nat. Einheiten
            #
            # quad ermöglicht es bis Unendlich zu integrieren, da die Stützstellen selbstständig ermittelt werden. Für große Massen ergaben sich in dieser Berechnung jedoch 
            # Schwierigkeiten in der BEstimmung besagter Stützstellen. Daher wurde die obere Grenze der Integrale für m>300 eV auf einen von der Masse Abhägngigen Wert beschränkt.
            # Es wird also nur noch über einen Berecih proportional zur effektiven Reichweite des Integrals berechnet, welches die BEstimmugn der Gitterpunkte vereinfacht und die
            # numerische Genauigkeit erhöht.
            if m_array_mint[iii]<=3e2:
               
                integral, err = quad(integrand_nat_units_without_c, 0, smp.oo, args=(n_array_paper_mint[ii], l_array_paper_mint[i], m_array_mint[iii]),epsrel=1e-60)

            if m_array_mint[iii]>3e2:
                r_max = 500/m_array_mint[iii]
                integral, err = quad(integrand_nat_units_without_c, 0, r_max, args=(n_array_paper_mint[ii], l_array_paper_mint[i], m_array_mint[iii]),epsrel=1e-60)

            # jeweiliges Speichern der Ergebnisse 
            c_m_data[i,ii,iii,0] = float(m_array_mint[iii])
            c_m_data[i,ii,iii,1] = np.abs(integral/Enl(n_array_paper_mint[ii]))


# print(c_m_data[0,0,:,1])


# Nun folgt die BErechnung der Integrale zur BErechnung der EVrscheibung auf den Grundzustand. Da diese erst im Nachhinein berechnet wurde, wurde eine neue Funktion defineirt.

# Die Integrale werden hier in einer Liste egscheichert, welches die Auswertung erleichtert. 
integrals_ground_list=[]

for m in m_array_mint:
    # Die Aufteilung des Integrationsintervalles folgt dem obrigen Beispiel.
    if m<=3e2:
        integrals_ground, err_ground = quad(integrand_nat_units_without_c, 0, smp.oo, args=(1, 0, m),epsrel=1e-60)
        integrals_ground_list.append(integrals_ground/Enl(1))
    else:
        r_max=500/m
        integrals_ground, err_ground = quad(integrand_nat_units_without_c, 0, r_max, args=(1, 0, m),epsrel=1e-60)
        integrals_ground_list.append(integrals_ground/Enl(1))
    

# print(integrals_ground_list)


# Zuletzt folgt die Berechnung der Kopplungskonstante für jede Bosonenmasse m.
# Es wird über jedes n und l iterriert, um für jeden Zustand die zugehörige Kopplungskonstante c in Abhängigkeit der Masse m zu berechnen.
# Dazu wird die Formel c=int(m)/10^(-12) verwendet. verwendet
for i in range(0,len(l_array_paper_mint)):
    for ii in range(0,len(n_array_paper_mint)):
        c=1e-12/c_m_data[i,ii,:,1]
        # print(c_m_data[i,ii,:,1])
        # print(c)

# Berechnung der Kopplungskonstante für den Grundzustand nach der Formel c=int(m)/10^(-12)
c_ground=1e-12/np.array(integrals_ground_list)

[2.16645130e+01 2.16587157e+01 2.16526886e+01 2.16464228e+01
 2.16399088e+01 2.16331371e+01 2.16260975e+01 2.16187796e+01
 2.16111725e+01 2.16032650e+01 2.15950453e+01 2.15865014e+01
 2.15776207e+01 2.15683900e+01 2.15587959e+01 2.15488243e+01
 2.15384607e+01 2.15276899e+01 2.15164962e+01 2.15048635e+01
 2.14927748e+01 2.14802128e+01 2.14671594e+01 2.14535957e+01
 2.14395025e+01 2.14248595e+01 2.14096460e+01 2.13938404e+01
 2.13774203e+01 2.13603627e+01 2.13426437e+01 2.13242384e+01
 2.13051213e+01 2.12852659e+01 2.12646448e+01 2.12432298e+01
 2.12209915e+01 2.11978999e+01 2.11739236e+01 2.11490304e+01
 2.11231871e+01 2.10963594e+01 2.10685118e+01 2.10396079e+01
 2.10096099e+01 2.09784793e+01 2.09461759e+01 2.09126587e+01
 2.08778854e+01 2.08418124e+01 2.08043950e+01 2.07655874e+01
 2.07253421e+01 2.06836109e+01 2.06403440e+01 2.05954904e+01
 2.05489981e+01 2.05008135e+01 2.04508821e+01 2.03991481e+01
 2.03455544e+01 2.02900428e+01 2.02325542e+01 2.01730280e+01
 2.01114030e+01 2.004761

In [ ]:
# analytische Lösung des Problems für zirkuläre Zustände in Python für die Kontrolle der numerischen Ergebnisse

# Es müssen die Konstanten übergeben werden, welche beliebig gewählt werden können. Sonst erwartet Python einen Zahlenwert für jede 
# Variable, welches für analytische Berechnungen nicht gewollt ist. In den Klammern werden Grenzen auf die Konsatnten gesetzt (ungleich 0 usw) um nur physikliche
# Lösungen zu erhalten. Diese sind für die Berechnung der Integrale notwendig.

# Kopplungskonstante c, Bohrradius a, Bosonenmasse m
c, a, m = smp.symbols("c a m", positive = True, real = True)
# Hauptquantenzahl n
n = smp.symbols("n", positive = True, integer = True, real = True)
# Integrationsvariable
rho = smp.symbols("rho", real = True, positive = True)

# Definiton der n_Quantenzahlen zur Auswertung
n_array=np.array([1,2,3])

# Definition des Integranten r^2*Rnl(rho)^2*V_NP(r) mit l=n-1 für zirkuläre Zustände
def f(rho,m,n,a,c):
    return -10.90497832*c/(4*np.pi)*1/(a*n**2*smp.factorial(2*n-1))*rho**(2*n-1)*smp.exp(-(1+(n*a*2.68172763*10**(-4)*m)/2)*rho)

#def f(rho,c,a_0,m,n):
#return -c/(4*np.pi)*1/(a_0*n**2*smp.factorial(2*n-1))*rho**(2*n-1)*exp(-(1+(2*m)/(n*a_0))*rho)

# analytische Integration über alle positiven reellen Zahlen.
# .simplify() vereinfacht den Ausdruck
def int_py(m,n,a,c):
    return smp.integrate(f(rho,m,n,a,c), (rho, 0, smp.oo)).simplify()

# Ausgabe der Energieverschiebungen für jedes n in _array
for n in n_array:
    Enl=-Rydberg_energy/n**2
    print("n = ",n, ",  l = ", n-1)
    print(Enl)
    print("Die absolute Verschiebung beträgt ", int_py(mass,n,1,coupling), "eV")
    print("Die relative Verschiebung zur Grobstrukturenergie beträgt ", int_py(mass,n,a_0,coupling)/Enl)

In [ ]:
# Einladen der Daten für die Wasserstoffübergänge aus CODATA 2022, siehe Abschnitt 3.1

filename = "wasserstoffübergänge_diff_zwischen_2_energieniveaus_codata_2022.txt"

# Datei einlesen mit pandas
df = pd.read_csv(filename, delimiter=",", skiprows=0, encoding="latin1", decimal=".")

# Konvertiere die relevanten Spalten zu int (außer 'messung', falls sie ein String ist)
# df.iloc[:, 1:] = df.iloc[:, 1:].astype(float)

# Extrahiere die Spalten als NumPy-Arrays. Da manche Datenpunkte aus der Differenz zweier Übergänge bestehen, besteht jeder Datenpunkt aus jeweils
# 4 n-, j- und l-Quantenzahlen. Für Datenpunkte, in denen nur ein Übergang betrachtet wird, werden die letzten beiden Triplets (n,j,l) auf 0 gesetzt.
# Dies gilt als Entscheidungskriterium in der späteren Anpassungsfunktion und ermöglicht die Analyse aller Datenpunkte mit nur einer Funktion.

# messpunkte gibt den Naemnd es Übergabnges ein
messpunkte = df.iloc[:, 0].values.astype(str)

# Einladen erstes Triplett
n1s = df.iloc[:, 1].values.astype(float)
j1s = df.iloc[:, 2].values.astype(float)
l1s = df.iloc[:, 3].values.astype(float)

# Einladen zweites Triplett
n2s = df.iloc[:, 4].values.astype(float)
j2s = df.iloc[:, 5].values.astype(float)
l2s = df.iloc[:, 6].values.astype(float)

# Einladen drittes Triplett
n3s = df.iloc[:, 7].values.astype(float)
j3s = df.iloc[:, 8].values.astype(float)
l3s = df.iloc[:, 9].values.astype(float)

# Einladen viertes Triplett
n4s = df.iloc[:, 10].values.astype(float)
j4s = df.iloc[:, 11].values.astype(float)
l4s = df.iloc[:, 12].values.astype(float)

# Einladen der Übergangsfrequenzen in kHz
freqs = df.iloc[:, 13].values.astype(float) 

# Einladen der Fehler auf die Frequenzen in kHz
freqs_errs = df.iloc[:, 14].values.astype(float) 

# Speichern aller Argumente und Frequenzen in einem Array, umd die Argumente der Anpassungsfunkiton übersichtlicher zu gestalten
x = np.array([n1s, j1s, l1s, n2s, j2s, l2s, n3s, j3s, l3s, n4s, j4s, l4s], dtype=float)

#----------------------------------------------------------------------------------------------------------------------------------------------------------------

# Nun folgt die BErechnung der theoretischen Abweichungen. An sich können diese seperat in jeder chi^2-Anpassung ermittelt werden, 
# jedoch treten diese immer in Kombination mit den Messunsicherheiten auf. Deswegen wird der GEsamtfehler über die quadratische Fehleraddition
# direkt für jeden Datenpuntk berechnet und gespeichert. 

# Definition der Fehlerfunktion, siehe Abschnitt 3.3
def th_err(n,l):
    if l==0:
        return 2.6/n**3 #error in kHz
    else:
        return 0

def error_calc(xdatas,freq_unc):
    n1=xdatas[0]
    l1=xdatas[2]
    n2=xdatas[3]
    l2=xdatas[5]
    n3=xdatas[6]
    l3=xdatas[8]
    n4=xdatas[9]
    l4=xdatas[11]
    
    theo_err = np.zeros(len(n3))

    for i in range(0,len(n3)):
        if n3[i]==0:
            theo_err[i] = th_err(n1[i],l1[i]) + th_err(n2[i],l2[i])
        else:
            theo_err[i] = th_err(n1[i],l1[i]) + th_err(n2[i],l2[i]) + 1/4*( th_err(n3[i],l3[i]) + th_err(n4[i],l4[i]) )
            
    total_error = (theo_err**2 + freq_unc**2)**(0.5) 
    
    return total_error

total_unc = error_calc(x,freqs_errs)
print("done")

done


In [ ]:
# Implimentierung der Anpassungsfunktion für die theoretischen Energien im Standartmodell, siehe Abschnitt 3.2 und CODATA 2022 
# Hinter jedem Term steht in Klammern die entsprechende Nummer der Gleichung in 
# CODATA Recommended Values of the Fundamental Physical Constants: 2022 (url: https://arxiv.org/abs/2409.03787).
# Bei den Arrays für numerische Korrekturen beziehen sich die Verweise auf den Tabellen auf die selbe Quelle.
#
# Für Zustände wird die Bezeichnung nl_j verwendet, wobei n die Hauptquantenzahl, l die Bahndrehimpulsquantenzahl und j die Gesamtspinquantenzahl ist.
# 
# Alle Energieterme sind in SI Einheiten anegegeben

# konstante in der Drei Photon korrektur 
a4 = 0.517479061 

# häufig auftretender Term in der Implimentierung
L = ln( (m_e/m_r) * ((z*alpha)**(-2)) ) 

# Definiton des Friarradius in der Kernkorrektur in Ordnung alpha^5
r_pF_fm = 1.947 #Friar radius in fm 
r_pF= r_pF_fm*1e-15 #Friar Radius in m 


# Array mit ausgewählten Werte für den Bethe-Logarithmus für n-Quantenzahlen (geben Zeile an und stehen in erster Spalte) 
# und l-Quantenzahlen (die i-te Spalte entspricht l=i-1, wobei i von 0 an gezählt wird).
# Null-Einträge sind sind relevant für die hier verwendteten Übergänge.
# Zu finden Tab V
bethe_log = np.array([[1,2.984128556,0,0],
                      [2,2.811769893,-0.030016709,0],
                      [3,2.767663612,0,0],
                      [4,2.749811840,-0.041954895,-0.006740939],
                      [5,0,0,0],
                      [6,2.735664207,0,-0.008147204],
                      [7,0,0,0],
                      [8,2.730267261,0,-0.008785043],
                      [9,0,0,0],
                      [10,0,0,0],
                      [11,0,0,0],
                      [12,0,0,-0.009342954]]) 


#G_SE(alpha) Funktion aus Tab VII, sp steht für spalte: sp0=n-Quantenzahlen, sp1=ns1/2, sp2=np1/2, sp3=np3/2, sp4=nD3/2, sp5=nD5/2
G_se_matrix = np.array([[1,-30.290240,0,0,0,0],
                 [2,-31.185150,-0.97350,-0.48650,0,0],
                 [3,-31.04770,0,0,0,0],
                 [4,-30.9120,-1.1640,-0.6090,0,0.03163],
                 [5,0,0,0,0,0],
                 [6,-30.711,0,0,0,0.03417],
                 [7,0,0,0,0,0],
                 [8,-30.606,0,0,0.007940,0.03484],
                 [9,0,0,0,0,0],
                 [10,0,0,0,0,0],
                 [11,0,0,0,0,0],
                 [12,0,0,0,0.009130,0.03512]
                ]) 

#G_VP^(1) funktion aus Tab VIII, sp steht für spalte: sp0=n-Quantenzahlen, sp1=ns1/2, sp2=np1/2, sp3=np3/2, sp4=nD3/2, sp5=nD5/2
G_vp_1_matrix = np.array([[1,-0.618724,0,0,0,0],
                 [2,-0.808872,-0.064006,-0.014132,0,0],
                 [3,-0.814530,0,0,0,0],
                 [4,-0.806579,-0.080007,-0.017666,0,0],
                 [5,0,0,0,0,0],
                 [6,-0.791450,0,0,0,0],
                 [7,0,0,0,0,0],
                 [8,-0.781197,0,0,0,0],
                 [9,0,0,0,0,0],
                 [10,0,0,0,0,0],
                 [11,0,0,0,0,0],
                 [12,0,0,0,0,0]
                ])

#B_60 Korrektur aus Tab VII Term für die Zwei-Photon Korrektur 
B_60_matrix= np.array([[1,-78.7,0,0,0,0],
                [2,-63.6,-1.8,-1.8,0,0],
                [3,-60.5,0,0,0,0],
                [4,-58.9,-2.5,-2.5,0,0.178],
                [5,0,0,0,0,0],
                [6,-56.9,0,0,0,0.207],
                [7,0,0,0,0,0],
                [8,-55.9,0,0,0.245,0.221],
                [9,0,0,0,0,0],
                [10,0,0,0,0,0],
                [11,0,0,0,0,0],
                 [12,0,0,0,0.259,0.235]
                ]) 


# Korrektur in E_rel_recoil_tp5 aus Tab VI mutipliziert mit pi
G_red_pi_matrix = np.array([[1,9.72,0,0],
                     [2,14.899,1.5097,-2.1333],
                     [3,15.242,0,0],
                     [4,15.115,0,0],
                     [5,14.941,0,0],
                     [6,14.8,0,0],
                     [7,0,0,0],
                     [8,14.7,0,0],
                     [9,0,0,0],
                     [10,0,0,0],
                     [11,0,0,0],
                     [12,0,0,0],
                    ]) 


# Korrekturterm in two Photon, Tab VI + Extrapollierung nach Jentschura 2003
N_nS_matrix = np.array([[1,17.85567203],
                 [2, 12.03214158],
                 [3, 10.449809],
                 [4, 9.722413],
                 [5,0],
                 [6, 9.031832],
                 [7,0],
                 [8,8.697639],
                 [9,8.21333333],
                 [10,8.1623],
                 [11,8.12181818], 
                 [12,8.08895833]
                ]) 
 
# Korrekturterm in two Photon, Tab VI
N_nP_matrix = np.array([[1,0],
                 [2,0.003300635],
                 [3,0],
                 [4,-0.000394332],
                 [5,0],
                 [6,0],
                 [7,0],
                 [8,0],
                 [9,0],
                 [10,0],
                 [11,0],
                 [12,0],
                ]) 

# Korrekturterm in two Photon, Tab VII
B_71_s_matrix = np.array([[1,-116],
                  [2,-100],
                  [3,-94],
                  [4,-91],
                  [5,0],
                  [6,-88],
                  [7,0],
                  [8,-86],
                  [9,0],
                  [10,0],
                  [11,0],
                  [12,0],
                ]) 

#angular momentum quantum parity number, siehe Dirac Energie Absch. III.A.1
def kappa(j,l): 
    return ( (-1)**(int(j+0.5)-l) ) * int(j+0.5)

# Gl. (29)
def f(n,j,l):
    return ( 1 + (z*alpha)**2 / ( (n - np.abs(kappa(j,l)) + ( (kappa(j,l)**2 - (z*alpha)**2) )**(0.5) )**2) )**(-0.5)

# Definiton der Digammefunktion
def psi(n):
    return digamma(n)

# Definiton der harmonischen Reihe
def sum_1_over_n(n):
    return sum(1 / j for j in range(1, int(n+1)))


# Gl(33) um Nullen im Nenner zu vermeiden
def a_n(n,l): #otherwise devision by 0
    if l==0:
        if n==0:
            return 0
        else: 
            return (-2)*ln(2/n) - 2 + 1/n - 2*sum_1_over_n(n)
    else:
        return 1/(l*(l+1)*(2*l+1))
    
# Term aus Gl(34) um Nullen im Nenner zu vermeiden
def recoil_tp6_1_over_l(l): 
    if l==0:
        return 0
    else:
        return 2 / ((2*l-1)*(2*l+1)*(2*l+3)) 
    

# Korrekturterm in E_self_energy, siehe Gl(35), um Nullen im Nenner zu vermeiden
def A_61_last_term(n,j,l): 
    if n==0:
        return 0
    
    elif l==0:
        return 0
    
    else:
        return (96*n**2 - 32*l*(l+1)) / ( 3*n**2*(2*l-1)*(2*l)*(2*l+1)*(2*l+2)*(2*l+3) )

# Korrekturterm in E_nuc_7, siehe Gl(62), um Nullen im Nenner zu vermeiden
def E_nuc_7_last_term(n,j,l): 
    if n==1:
        return 0
    else:
        return 4 * n**2 / (n**2 - 1) * N_nP_matrix[int(n)-1,1]
            

#----------------------------------------------------------------------------------------------

# Die folgenden Funktionen sind die Implementierung der Energieverschiebungen in nat. Einheiten, siehe Abschnitt 3.2 und CODATA 2022

# Gl(30)
def E_dirac(n,j,l,Rydberg):
    return (
            (f(n,j,l) - 1) *2*const.h*Rydberg*const.c/(alpha**2)*m_r/m_e
            - (f(n,j,l) - 1)**2 *const.h*Rydberg*const.c/(alpha**2)*m_p*m_r/M**2 + 
            (1 - delta(l,0)) / (kappa(j,l)*(2*l + 1)) *(z*alpha)**4 *m_r**3*const.c**2 /(2*n**3*m_p**2)
            )
    
# Gl(32)
def E_rel_recoil_tp5(n,j,l): 
    return (
            (m_r**3) / ((m_e**2)*m_p) *(z*alpha)**5 / (np.pi*(n**3)) *m_e*(const.c**2)* 
            (
            1/3*delta(0,l) *ln((z*alpha)**(-2)) - 8/3*bethe_log[int(n)-1,int(l)+1] - 1/9*delta(0,l) - 7/3*a_n(n,l)
            - 2 / ((m_p**2) - (m_e**2)) *delta(0,l)*( m_p**2*ln(m_e/m_r) - (m_e**2)*ln(m_p/m_r) )
            ))  

# Gl(34)
def E_rel_recoil_tp6(n,j,l): 
    return (((z*alpha)**6) / (n**3)* m_e/m_p *m_e*(const.c**2)*
            (
             (4*ln(2) - 7/2)*delta(0,l) +  (3 - l*(l+1)/(n**2)) * recoil_tp6_1_over_l(l)
            + (z*alpha)*G_red_pi_matrix[int(n)-1,int(l)+1]/np.pi 
            ) 
            ) 

# Gl(35,36)
def E_self_energy(n,j,l): 

    A41 = 4/3*delta(0,l)
    
    A40 = - 4/3*bethe_log[int(n)-1,int(l)+1] + (10/9)*delta(0,l)- m_e/m_r / (2*kappa(j,l)*(2*l+1)) *(1 - delta(0,l))
    
    A50 = (139/32 - 2*ln(2))*np.pi*delta(0,l)
    
    A62 = - delta(0,l)
    
    A61 = ( (4*sum_1_over_n(n) + 28/3*ln(2) - 4*ln(n) - 601/180 - 77 / (45*n**2) )*delta(0,l) + 
                (n**2 - 1) / (n**2) *( 2/15 + 1/3*delta(j,0.5) )*delta(1,l) 
                + A_61_last_term(n,j,l) )
    
    # Gl(36)
    F = A41*L + A40 + (z*alpha)*A50 + (z*alpha)**2*( A62* (L**2) + A61*L + G_se_matrix[int(n)-1 , int(float(l)+float(j)+0.5)])
    
    #Gl(36)
    return ( (alpha / np.pi) *((z*alpha)**4) / (n**3) *((m_r/m_e)**3) *F *m_e*(const.c**2) 
            )  

# Gl(37-41)
def E_vac_pol_stat_nuc(n,j,l): 
    
    V40 = - 4/15*delta(0,l)

    V50 = 5*np.pi / 48 *delta(0,l)

    V61 = - 2/15*delta(0,l)

    # Gl (39)
    H1 = V40 + (z*alpha) *V50 + (z*alpha)**2*(V61*L + G_vp_1_matrix[int(n)-1,int(int(l)+float(j)+0.5)])    

    # Gl (41)
    GVPR = ( 19/45 - np.pi**2 / 27 + (1/16 - (31*np.pi**2)/2880)*np.pi*z*alpha )*delta(0,l)

    # Gl(40)
    HR = (z*alpha)**2 *GVPR

    # Gl (38)
    H = H1 + HR

    # Gl(37)
    return (
            alpha / np.pi *((z*alpha)**4) / (n**3) *((m_r/m_e)**3)* H *m_e*(const.c**2)
            )

# Gl(42)
def E_vac_pol_my(n,j,l): 
    return alpha / np.pi *((z*alpha)**4) / (n**3) *( -4/15*delta(0,l) )*((m_e/m_my)**2)*((m_r/m_e)**3)*m_e*(const.c**2) 

# Gl(43)
def E_vac_pol_had(n,j,l): 
    return 0.671*E_vac_pol_my(n,j,l)
       
# Gl(44)-(46)
def E_two_photon(n,j,l): 

    B40 = ( 3*(np.pi**2) / 2 *ln(2) - 10*(np.pi**2) / 27 - 2179/648 - 9 / 4 *zeta(3) ) *delta(0,l) + (
    (np.pi**2)*ln(2)/2 - (np.pi**2)/12 - 197/144 - 3*zeta(3)/4)* (m_e/m_r) *(1 - delta(0,l))/(kappa(j,l)*(2*l + 1))
    
    B50 = - 21.55447*delta(0,l)

    B63 = - 8 / 27*delta(0,l)

    B62 = 16/9* ( 71/60 - ln(2) + psi(n) + gamma - ln(n) - 1/n + 1/(4*(n**2)) )*delta(0,l) + 4/27*((n**2)-1) / (n**2)*delta(1,l)
    
    B61 = ( 413581/64800 + 4*N_nS_matrix[int(n)-1,1] / 3 + 2027*(np.pi**2)/864 - 
                616*ln(2)/135 - 2*(np.pi**2)*ln(2)/3 + 40*(ln(2))**2 / 9 + zeta(3) 
                + ( 304/315 - 32*ln(2)/9 ) * ( 3/4 + gamma + psi(n) - ln(n) - 1/n + 1/(4*(n**2)) ) 
                - 43/36 + 709*(np.pi**2)/3456 
                ) *delta(0,l) + ( 
                4 / 3 *N_nP_matrix[int(n)-1,1] + ((n**2) - 1) / (n**2) *( 31/405 + 1/3*delta(0.5,j) - 8/27*ln(2) ) 
                )*delta(1,l) 
    
    #-----------

    B72 = ( - 427 / 144 + 4*ln(2)/3 )*np.pi*delta(0,l)

    B71 = B_71_s_matrix[int(n)-1,1]*delta(0,l) + np.pi*( 427 / 432 - 4*ln(2)/9 )*(1 - 1/(n**2))*delta(1,l) 

    B60 = B_60_matrix[int(n)-1,int(int(l)+float(j)+0.5)]

    # Gl (46)
    G4 = B60 + (z*alpha)*(B72* (ln(((z*alpha)**(-2))))**2 + B71 *ln((z*alpha)**(-2)))

    #-------------

    # Gl (45)
    F = B40 + (z*alpha)*B50 + (z*alpha)**2*( B63 * L**3 + B62*L**2 + B61*L + G4)

    # Gl (44)
    return (
            (alpha/np.pi)**2 * ((z*alpha)**4)/(n**3)*((m_r/m_e)**3)*F * m_e*(const.c**2)
            ) 

# Gl(47), (48)
def E_three_photon(n,j,l):

    C40 = (- 568*a4 / 9 + 85*zeta(5)/24 - 121*(np.pi**2)*zeta(3)/72 - 84071*zeta(3)/2304 
            - 71*(ln(2))**4 / 27 - 239*(np.pi**2)*(ln(2))**2 / 135 + 4787*(np.pi**2)*ln(2) / 108 
            + 1591*(np.pi**4) / 3240 - 252251*(np.pi**2)/9720 + 679441/93312
            ) *delta(0,l) + (
            - 100*a4 / 3 + 215*zeta(5) / 24 - 83*(np.pi**2)*zeta(3) / 72 - 139*zeta(3)/18 
            - 25*(ln(2)**4) / 18 + 25*(np.pi**2)*(ln(2)**2) / 18 + 298*(np.pi**2)*ln(2) / 9 
            + 239*(np.pi**4)/2160 - 17101*(np.pi**2) / 810 - 28259 / 5184
            )*(m_e/m_r) *( 1-delta(0,l) )/ (kappa(j,l)*(2*l+1)) 

    C50 = - 3.3 * delta(0,l)

    C63 = 0

    C62 = - 2/3*(
            -2179/648 - 10*(np.pi**2)/27 + 3 / 2 *(np.pi**2)*ln(2) - 9 / 4 *zeta(3)
            )*delta(0,l)

    C61 = delta(1,l)* 2 / 9 * (n**2 - 1)/(n**2) *(
            - 2179/648 - 10*(np.pi**2) / 27 + 3/2*(np.pi**2)*ln(2) - 9/4*zeta(3) 
            )

    C60 = 0

    # Gl (48)
    F = C40 + (z*alpha)*C50 + (z*alpha)**2*(C63*L**3 + C62*L**2 + C61*L + C60)

    # Gl (47)
    return  (
            ((alpha / np.pi)**3) *((z*alpha)**4) / (n**3) *(m_r/m_e)**3 * F *m_e*(const.c**2) 
            )       


# Gl (50)
def E_nuc_4(n,j,l,prot_rad): 
    return 2/3*m_e*(const.c**2) *((z*alpha)**4)/(n**3)* ((m_r/m_e)**3) *((prot_rad/comp_wave_red)**2) *delta(0,l) 
          

# Gl (51)
def E_nuc_5(n,j,l): 
    return - 1/3*m_e*(const.c**2)*((z*alpha)**5) / (n**3) *((m_r/m_e)**3) *((r_pF/comp_wave_red)**3) *delta(0,l) 

# Gl (56)-(60)
def E_nuc_6(n,j,l,prot_rad):
    
    # Gl (59)
    rN2 = 1.068497*prot_rad

    # Gl (56)
    Efns = (m_e*const.c**2 *((z*alpha)**6) / (n**3)* ((m_r/m_e)**3)* ((prot_rad / comp_wave_red)**2) *
            (
            (-2/3)* (
            9 / (4*(n**2)) - 3 - 1/n + 2*gamma - ln(n/2) + psi(n) + 
            ln(m_r/m_e * rN2 / comp_wave_red *z*alpha) 
            ) *delta(0,l) + 1/6*(1 - 1 / (n**2)) *delta(kappa(j,l),l) 
            ))
    
    # Gl (57)
    Epol =  0.393 *delta(0,l) / (n**3)  * const.h * 1e3

    # Gl (60)
    Erad = 2/3*m_e*(const.c**2) *alpha*((z*alpha)**5) / (n**3) *((m_r/m_e)**3) *((prot_rad/comp_wave_red)**2) *(4*ln(2) - 5) *delta(0,l)

    return  Efns + Epol + Erad


# Gl (61), (62)
def E_nuc_7(n,j,l,prot_rad): 

    # Gl (61   )
    Enuc7_nS = 2/3*m_e*const.c**2 * alpha*(z*alpha)**6/(np.pi*n**3)*(m_r/m_e)**3*(prot_rad/comp_wave_red)**2*(
            -2/3*( ln((z*alpha)**(-2)) )**2 
            + ( ln(m_r/m_e* prot_rad/(comp_wave_red)) )**2
            )*delta(0,l)
    
    # Gl (62)
    Enuc7_nP = (1/6*m_e*(const.c**2) *alpha*((z*alpha)**6) / (np.pi*n**3) *((m_r/m_e)**3) *((prot_rad/comp_wave_red)**2)*(1 - 1/(n**2)) *(
            8/9*ln((z*alpha)**(-2)) 
            - 8/9*ln(2) + 11/27 + delta(kappa(j,l),l) + E_nuc_7_last_term(n,j,l)))*delta(l,1)
    

    return Enuc7_nS + Enuc7_nP


# Gl (63)
def E_rad_rec(n,j,l): 
    return ((m_r**3) / ((m_e**2)*m_p) *(alpha*((z*alpha)**5)) / ((np.pi**2)*(n**3)) *m_e*(const.c**2)*delta(0,l)*
            ( 
            6*zeta(3) - 2*np.pi**2*ln(2) + 35*(np.pi**2)/36 - 448/27 
            + 2/3*np.pi*(z*alpha)*( ln((z*alpha)**(-2)) )**2  
            )
            )      

# Gl (64)
def E_nuc_self_en(n,j,l): 
    return 4*(z**2)*alpha*((z*alpha)**4) / (3*np.pi*(n**3)) *(m_r**3)/(m_p**2)*(const.c**2)*( 
            ln(m_p / (m_r*((z*alpha)**2)) ) *delta(0,l) - bethe_log[int(n)-1,int(l)+1])
              
# Gesamtenergie in J
def E_tot(n,j,l,Rydberg,prot_rad): 
    return E_dirac(n,j,l,Rydberg) + E_rel_recoil_tp5(n,j,l) + E_rel_recoil_tp6(n,j,l) +E_self_energy(n,j,l) + E_vac_pol_stat_nuc(n,j,l) + E_vac_pol_my(n,j,l) +E_vac_pol_had(n,j,l) + E_two_photon(n,j,l) + E_three_photon(n,j,l) +E_nuc_4(n,j,l,prot_rad) +E_nuc_5(n,j,l) +E_nuc_6(n,j,l,prot_rad) +E_nuc_7(n,j,l,prot_rad) +E_rad_rec(n,j,l) + E_nuc_self_en(n,j,l)

# Frequenz eines Energiewertes in kHz
def f_total(n,j,l,Rydberg,prot_rad): 
    return E_tot(n,j,l,Rydberg,prot_rad)/const.h*1e-3


print("done")

done


In [ ]:
# Definition der Anpassungsfunktion im Stadardmodell.
# Diese wird in drei Teile unterteilt, die sich auf die Abhängigkeit von Rydberg, Protonradius und den Rest beziehen.

# Rydberg-Abhängigder Term
def freq_Ry_dep(n,j,l,Rydberg):
    result = E_dirac(n,j,l,Rydberg)/(const.h)
    return result

# Protonradiusabhängiger Term
def freq_Rp_dep(n,j,l,prot_rad):
    result = ( E_nuc_4(n,j,l,prot_rad) + E_nuc_6(n,j,l,prot_rad) + E_nuc_7(n,j,l,prot_rad) )/(const.h)
    return result

# Restterm
def freq_SM_rest(n,j,l):
    result = ( E_rel_recoil_tp5(n,j,l) + E_rel_recoil_tp6(n,j,l) + E_self_energy(n,j,l) + E_vac_pol_stat_nuc(n,j,l) + E_vac_pol_my(n,j,l) +E_vac_pol_had(n,j,l) + E_two_photon(n,j,l) + E_three_photon(n,j,l) + E_nuc_5(n,j,l) + E_rad_rec(n,j,l) + E_nuc_self_en(n,j,l) )/const.h
    return  result

# Gesamtfrequnz des Zustandes in Hz
def freq_tot_SM(n,j,l,Rydberg,prot_rad):
    return freq_Ry_dep(n,j,l,Rydberg) + freq_Rp_dep(n,j,l,prot_rad) + freq_SM_rest(n,j,l)


# eigentliche Anpassungsfunktion, xdatas sind die Argumente der Daten, Rydberg ist die Rydbergkonstante und prot_radius der Protonradius
def freq_SM_fit(xdatas,Rydberg,prot_radius): 
    # Entpacken der Argumente
    n1=xdatas[0]
    j1=xdatas[1]
    l1=xdatas[2]
    n2=xdatas[3]
    j2=xdatas[4]
    l2=xdatas[5]
    n3=xdatas[6]
    j3=xdatas[7]
    l3=xdatas[8]
    n4=xdatas[9]
    j4=xdatas[10]
    l4=xdatas[11]
    # print(f"prot_rad = {log_prot_rad:.6e}, Rydberg = {Rydberg:.4e}")
    
    # Definition des Ergebnisarrays
    result = np.zeros(len(n3))
    
    # Unterscheiden, ob ein Übergang oder die Differenz zweier Übergänge vorliegt nach der MEthode, welche beim Einladen der Daten erläutert wurde
    for i in range(0,len(n3)):
        if n3[i]==0:
            result[i] = freq_tot_SM(n1[i],j1[i],l1[i],Rydberg,prot_radius) - freq_tot_SM(n2[i],j2[i],l2[i],Rydberg,prot_radius)
        else:
            result[i] = freq_tot_SM(n1[i],j1[i],l1[i],Rydberg,prot_radius) - freq_tot_SM(n2[i],j2[i],l2[i],Rydberg,prot_radius) - 1/4*( freq_tot_SM(n3[i],j3[i],l3[i],Rydberg,prot_radius) - freq_tot_SM(n4[i],j4[i],l4[i],Rydberg,prot_radius) )
    
    # Kontrolle, ob Undendlichkeiten auftreten. In diesem Fall wird eine Warnung ausgegeben mit den entsprechenden Werten für die Rydbergkonstante und den Protonradius
    if np.any(np.isnan(result)) or np.any(np.isinf(result)):
        print("Warnung: freq_SM_fit gibt ungültige Werte zurück!")
        print("Parameter: Rydberg =", Rydberg, ", Protonradius =", prot_radius)

    # Ausgabe der Frequenzen in kHz
    return (-1)*result*1e-3 

print("done")

done


In [ ]:
#Durchführung der 2D-Optimierung im Standardmodell


# chi^2-Funktion mit dem dekadischen Log von R_p als Parameter. Das Argument ist ein Array der Form [Rydberg, log10_Rp]
def chi2(params):

    # Entpacken der Parameter
    Ryd, log10_Rp = params
    Rp = 10 ** log10_Rp

    # Absicherung gegen unphysikalische oder instabile Werte
    if Rp <= 0 or not np.isfinite(Rp):
        return 1e20
    try:
        # Berechnung der Frequenzen für die gegebenen Parameter
        model = freq_SM_fit(x, Ryd, Rp)
        # Ausgabe der Rediduen
        return np.sum(((freqs - model) / total_unc) ** 2)
    
    except Exception as e:
        print("Fehler in freq_SM_fit:", e)
        return 1e20



# Festlegen der Startwerte für [Rydberg, log10_Rp]
initial_guess = [1e7, -15.042]

# Grenzsetzung für [Rydberg, log10_Rp]
bounds=[(0.5e7,1.5e7),(-15.3, -14.7)]



# Minimierung der chi^2-Funktion mit dem Powell-Algorithmus
result = minimize(chi2, initial_guess, bounds = bounds, method='Powell')

# Speichern 
# der optimalen Parameter
best_Ryd, log10best_Rp = result.x
best_Rp = 10**log10best_Rp
# des minimalen chi^2
min_chi2 = result.fun
# und des reduzierten chi^2, dof sind die "Degrees of freedom"
ndof = len(freqs) - len(result.x)
red_chi2 = min_chi2 / ndof

# Ausgabe der Ergebnisse
print("Optimale Parameter:")
print(f"Rydbergkonstante: {best_Ryd:.8f} m^-1, Abweichung zu CODATA= {((best_Ryd-Rydberg_CODATA)/Rydberg_CODATA)*100}%")
print(f"Protonradius: {best_Rp:.5e} m, Abweichung zu CODATA= {((best_Rp-r_N_CODATA)/r_N_CODATA)*100}%")
print(f"Minimales chi2: {min_chi2:.3f}")
print(f"Reduziertes chi2: {red_chi2:.3f}")

#-------------------------------------------------------------------------------------------------------------------------------------

# Fehlerabschätzung für den Protonradius wie in Abschnitt 3.4.2 beschrieben bei fixierter Rydbergkonsatnte
# Aufspannen eines Gitters mit Stützstellen, für welches die chi^2-Werte berechnet werden
logRp_range = np.linspace(log10best_Rp - 1, log10best_Rp + 1, 4000)
chi2_vals = np.array([chi2([best_Ryd, logRp]) for logRp in logRp_range])

# Interpolation zwischen den Stützstellen zur genaueren Abschätzung der Standartfehler
chi2_interp = interp1d(logRp_range, chi2_vals - min_chi2)


# Finde Grenzen bei chi² = min + 1
try:
    # Finden des ersten und des letzten Wertes von chi^2, welches kleiner als min + 1 ist
    lower_idx = np.where((chi2_vals - min_chi2) < 1)[0][0]
    upper_idx = np.where((chi2_vals - min_chi2) < 1)[0][-1]
    # BEstimmung der zugehörigen log10(Rp)-Werte für die Grenzen
    logRp_low = logRp_range[lower_idx]
    logRp_high = logRp_range[upper_idx]

    # Berechnung der besten log10(Rp)-Werte und der Unsicherheit
    Rp_low = 10 ** logRp_low
    Rp_high = 10 ** logRp_high
    Rp_best = 10 ** log10best_Rp
    sigma_Rp = (Rp_high - Rp_low) / 2

    # Ausgabe der Standartabweichungen
    print("\nProfilbasierte Fehlerabschätzung:")
    print(f"R_p = {Rp_best:.3e} ± {sigma_Rp:.2e} m")
    # Ausrechnen der Abweichung zum CODATA-Wert in Einheiten der zuvor berechneten Standartabweichungen
    print(f"{(Rp_best-r_N_CODATA)/sigma_Rp:.4f} Standartabweichung zum CODATA Wert")
except:
    print("Fehlerabschätzung über Profil nicht möglich.")


# Profil für Rydbergkonstante bei fixiertem log10(R_p) wie in Abschnitt 3.4.2 beschrieben
# Aufspannen eines Gitters mit Stützstellen, für welches die chi^2-Werte berechnet werden
Ryd_range = np.linspace(best_Ryd - 20, best_Ryd + 20, 4000)
chi2_vals_Ryd = np.array([chi2([Ryd, log10best_Rp]) for Ryd in Ryd_range])
chi2_shifted_Ryd = chi2_vals_Ryd - min_chi2


# Interpolation zwischen den Stützstellen zur genaueren Abschätzung der Standartfehler
chi2_interp = interp1d(Ryd_range, chi2_shifted_Ryd, kind='cubic', fill_value='extrapolate')

# Finde Grenzen bei chi² = min + 1 mit brentq, welches eine Nullstelle findet für ein 
# Intervall mit wechselndem Vorzeichen, da das Gitterverfahren keine zuverlässigen Werte lieferte
try:
    # Linke Grenze
    Ryd_left = brentq(lambda R: chi2_interp(R) - 1.0, Ryd_range[0], best_Ryd)
    # Rechte Grenze
    Ryd_right = brentq(lambda R: chi2_interp(R) - 1.0, best_Ryd, Ryd_range[-1])

    # Berechnen der Unsicherheit
    sigma_Ryd = (Ryd_right - Ryd_left) / 2
    # Ausgabe der Unsicherheit und Abweichung zum CODATA-Wert
    print(f"\nRydbergkonstante: {best_Ryd:.8f} ± {sigma_Ryd:.8f} m⁻¹ (Profil-Likelihood)")
    print(fr"{(best_Ryd-Rydberg_CODATA)/sigma_Ryd:.4f} Standartabweichungen zum CODATA Wert")
except Exception as e:
    print("Profilgrenzen für Rydberg nicht bestimmbar:", e)

Optimale Parameter:
Rydbergkonstante: 10973731.56863185 m^-1, Abweichung zu CODATA= 4.327182947477036e-09%
Protonradius: 8.91856e-16 m, Abweichung zu CODATA= 6.078658184287032%
Minimales chi2: 78.410
Reduziertes chi2: 4.356

Profilbasierte Fehlerabschätzung:
R_p = 8.919e-16 ± 5.14e-19 m
99.5212 Standartabweichung zum CODATA Wert

Rydbergkonstante: 10973731.56863185 ± 0.00000502 m⁻¹ (Profil-Likelihood)
94.6834 Standartabweichungen zum CODATA Wert


In [ ]:
# Berechnung der Energieverschiebung für die NP-Korrektur für jeden Zustand aus den Übergängen von CODATA
# Das Vorgehen wird in Abschnitt 3.5 beschrieben

# Bestimmung der MAssepuntke, für die die Kopplungskonsatnte verstimmt werden soll
mass_array_for_NP_fit=np.logspace(0,4,30)

# Der Cache wird definiert, auskommentieren setzt den Cache zurück, welches nur notwendig ist für die Neuberechnung aller Integrale um die GEschwindigkeit des Codes zu erhöhen
# NP_cache_integrals = {}


# Definieren der Integralfunktionen für die NP-Korrektur. Eingabeparameter sind die Quantenzahlen n,l 
# und die Masse des NP, sowie den Cache, in die die Korrekturen gespeichert werden sollen
def freq_tot_NP_no_c_integrals(n,l,mass,integral_cache):
    
    # Iteration über alle m, n und l-Werte
    for j in range(len(mass)):
        # print(mass_array_for_NP_fit[j])

        for i in range(len(n)):
            # Festlegung des Keys, wobei -1 ein repulsives und +1 ein attraktives Potential beschreibt. 
            # Beide werden parallel berechnet, damit diese Codezelle nur ein Mal ausgeführt werden muss.
            key_attr = (-1,float(n[i]),float(l[i]),float(mass[j]))
            key_rep = (1,float(n[i]),float(l[i]),float(mass[j]))

            # In den Daten befinden sich Null-Einträge, welche für die VErwendung von nur einer Anpassungsfunktion notwendig sind (siehe Einladen der Daten).
            # Diese werden hier übersprungen, da sie keine physikalische Bedeutung haben.
            if n[i] == 0:
                integral_cache[key_attr] = 0
                integral_cache[key_rep] = 0

            # Berechnung der Verscheibung für physikalische Zustände.
            else:
                result_attr, err_attr = quad(integrand_nat_units_without_c_attractive, 0, smp.oo, args=(n[i], l[i], mass[j]),epsrel=1e-20)  # Energie in eV
                integral_cache[key_attr] = result_attr*const.e/const.h*1e-3 #in kHz
                
                result_rep, err_rep = quad(integrand_nat_units_without_c_repulsive, 0, smp.oo, args=(n[i], l[i], mass[j]),epsrel=1e-20)  # Energie in eV
                integral_cache[key_rep] = result_rep*const.e/const.h*1e-3 #in kHz

    # Ausgabe der Frequenzverschiebung in kHz
    return integral_cache

# Alle n- und l-Quantenzahlen werden in einem Array gespeichert, um die Berechnung der Frequenzverschiebung zu beschleunigen
n_all_comb =[n1s, n2s, n3s, n4s]
l_all_comb = [l1s, l2s, l3s, l4s]

# Berechnung der Frequenzverschiebung für alle n- und l-Quantenzahlen
for i in range(0,3):
    freq_tot_NP_no_c_integrals(n_all_comb[i], l_all_comb[i], mass_array_for_NP_fit, NP_cache_integrals)

print("done")

done


In [ ]:
# Erweiterung der Anpassungsfunktion um die neue Wechselwirkung

# Bestimmung der Energieverscheibugn zum Zustand nl_j durch den Vergleich des Keys mit dem Cache, siehe vorherige Codezelle
def freq_tot_NP_without_c(n,j,l,m,kind):        
    key = (kind, int(n), int(l), float(m))
    res = NP_cache_integrals[key]
    return res
    
# Kombinierung der SM und NP Beiträge für einen Zustand
def freq_tot_SM_NP(n,j,l,Rydberg,prot_rad,c,m,kind):
    return freq_tot_SM(n,j,l,Rydberg,prot_rad) + c*freq_tot_NP_without_c(n,j,l,m,kind)
    

# Erweitere Anpassungsfunktion mit SM und NP-Beiträge.
# xdatas sind die Argumente der Daten, Rydberg ist die Rydbergkonstante und prot_radius der Protonradius.
# coup ist die Kopplungskonstante und mass die Masse des NP und kind die Art des Potentials (attraktiv oder repulsiv).
# 
def freq_SM_NP_fit(xdatas,Rydberg,prot_radius,coup,mass,kind): 
    # Entpacken der Argumente
    n1=xdatas[0]
    j1=xdatas[1]
    l1=xdatas[2]
    n2=xdatas[3]
    j2=xdatas[4]
    l2=xdatas[5]
    n3=xdatas[6]
    j3=xdatas[7]
    l3=xdatas[8]
    n4=xdatas[9]
    j4=xdatas[10]
    l4=xdatas[11]

    # print(coup)
    # print(f"prot_rad = {log_prot_rad:.4e}, Rydberg = {Rydberg:.4e}, mass = {mass:.4e}, coupling = {coup:.4e}")

    # Definition des Ergebnisarrays
    result = np.zeros(len(n3))
    
    # Iteration über alle n- und l-Quantenzahlen nach dem gleichen Prinzip wie in der SM-Anpassungsfunktion
    for i in range(0,len(n3)):
        if n3[i]==0:
            result[i] = freq_tot_SM_NP(n1[i],j1[i],l1[i],Rydberg,prot_radius,coup,mass,kind) - freq_tot_SM_NP(n2[i],j2[i],l2[i],Rydberg,prot_radius,coup,mass,kind)
        else:
            
            result[i] = freq_tot_SM_NP(n1[i],j1[i],l1[i],Rydberg,prot_radius,coup,mass,kind) - freq_tot_SM_NP(n2[i],j2[i],l2[i],Rydberg,prot_radius,coup,mass,kind) - 1/4*( freq_tot_SM_NP(n3[i],j3[i],l3[i],Rydberg,prot_radius,coup,mass,kind) - freq_tot_SM_NP(n4[i],j4[i],l4[i],Rydberg,prot_radius,coup,mass,kind) )
     # Ausgabe der tehoretischen Übergangsfrequenzen in kHz
    return (-1)*result*1e-3 

print("done")

done


In [ ]:
# 3D-Optimierung von (Rydberg, Protonradius, Kopplungskonstante) zur Bestimmung der oberen Grenzen auf die Kopplungskonstante für ein attraktives Potential
# Rydberg ist die Rydbergkonstante, Rp der Protonradius und coup bzw g die Kopplungskonstante
# --- Chi²-Funktion ---
def chi2_R_g_3D(params, xdatas, ydata, yerr, mass, kind):
    Rydberg, Rp, coup = params
    model = freq_SM_NP_fit(xdatas, Rydberg, Rp, coup, mass, kind)
    if not np.all(np.isfinite(model)):
        return 1e30
    return np.sum(((model - ydata) / yerr) ** 2)

# --- Parameter ---
mass_array = np.array(mass_array_for_NP_fit)
#attraktives Potential
kind = -1

# --- Ergebnis-Listen ---
# Kopplung
g_limit_list_3D_att = [] 
#chi^2_min
chi2_list_3D_att = []
# chi^2_red 
chi2_red_list_3D_att = []
# Rydberg
R_fit_list_3D_att = []
# Protonradius
Rp_fit_list_3D_att = []


# --- Freiheitsgrade: 3 Parameter (Rydberg, Rp, coup) ---
dof = len(freqs) - 3  # Anzahl Datenpunkte minus Anzahl freier Parameter

# --- Iteration über alle Massen ---
for mX in mass_array_for_NP_fit:
    # Initialwerte für Fit
    R_guess = best_Ryd
    g_guess = 1e-10
    Rp_guess = best_Rp

    # Fit: gleichzeitige Optimierung von Rydberg, Rp und g mit dem Powell-Algorithmus
    res = minimize(
        chi2_R_g_3D,
        x0=[R_guess, Rp_guess, g_guess],
        args=(x, freqs, total_unc, mX, kind),
        bounds=[(R_guess * 0.997, R_guess * 1.003),(Rp_guess*0.997, Rp_guess*1.003), (1e-13, 1e-7)],
        method='Powell'
    )
    # Ausgabe der Ergebnisse zur Kontrolle
    # print (res)
    
    # nan als Ausgabe bei dem Scheitern der Optimierung
    if not res.success:
        g_limit_list_3D_att.append(np.nan)
        chi2_list_3D_att.append(np.nan)
        chi2_red_list_3D_att.append(np.nan)
        R_fit_list_3D_att.append(np.nan)
        Rp_fit_list_3D_att.append(np.nan)
        continue

    # Entpacken der Ergebnisse
    R_fit, Rp_fit, g_fit = res.x
    chi2_min = res.fun
    # print(chi2_min)
    chi2_red = chi2_min / dof

    # Anhängen der optimalen Parameter in die Listen
    R_fit_list_3D_att.append(R_fit)
    Rp_fit_list_3D_att.append(Rp_fit)
    chi2_list_3D_att.append(chi2_min)
    chi2_red_list_3D_att.append(chi2_red)

    # # --- Gridsuche für g ---
    # # Aufspannen eines Gitters mit Stützstellen, für welches die chi^2-Werte berechnet werden
    # g_grid = np.logspace(-14, -4, 200)
    # # Berechnung der chi^2-Werte für die Stützstellen
    # chi2_grid = [chi2_R_g([R_fit, Rp_fit, g], x, freqs, total_unc, mX, kind) for g in g_grid]

    # # Bestimmung der 1σ-Schwelle
    # chi2_threshold = chi2_min + 1.0
    # # Bestimmung der Werte für g, die die Schwelle überschreiten
    # valid_g = g_grid[np.array(chi2_grid) <= chi2_threshold]

    # # Bestimmung der Grenzen für g
    # if len(valid_g) > 0:
    #     g_limit = np.max(valid_g)
    # else:
    #     g_limit = g_grid[-1] 

    #------------------------------------------------------------------------------------------------------------------------

    # Gridsuche mit Interpolation für g

    #Aufspannen eines Gitters mit Stützstellen, für welches die chi^2-Werte berechnet werden
    g_grid = np.logspace(-14, -5.1, 200)
    # Berechnung der chi^2-Werte für die Stützstellen
    chi2_grid = [chi2_R_g_3D([R_fit, Rp_fit, g], x, freqs, total_unc, mX, kind) for g in g_grid]
    chi2_target = chi2_min + 1.0

    # Bestimmung der 1σ-Schwelle
    try:
        # Interpolation der chi²-Werte
        chi2_interp = interp1d(g_grid, np.array(chi2_grid) - chi2_target, kind='linear')

        # Nullstelle finden im Bereich, in dem chi² erstmals die Schwelle überschreitet mit dem Nullstellenalgorithmus brentq
        # Suche nach dem ersten Punkt, an dem die Interpolation die Schwelle überschreitet
        for i in range(1, len(g_grid)):
            if (chi2_grid[i-1] - chi2_target) * (chi2_grid[i] - chi2_target) < 0:
                g_limit = brentq(chi2_interp, g_grid[i-1], g_grid[i])
                break
        # Wenn keine Nullstelle gefunden wurde, setze g_limit auf den letzten Punkt des Gitters, sodass trotzdem ein Wert zurückgegeben wird
        else:
            g_limit = g_grid[-1]  # fallback

    except (ValueError, RuntimeError):
        g_limit = g_grid[-1]  # fallback bei Interpolationsfehlern

    # -----------------------------------------------------------------------------------------------------------------------

    # # Wurzelbestimmung mit minimize_scaler bei festem R_p und Rydberg
    # # 1σ-Schwelle
    # chi2_threshold = chi2_min + 1.0

    # # Zielfunktion: negativ von g (weil wir maximieren wollen)
    # def g_objective(g):
    #     chi2_val = chi2_R_g([R_fit, Rp_fit, g], x, freqs, total_unc, mX, kind)
    #     return -g #if chi2_val <= chi2_threshold else np.inf

    # res_g = minimize_scalar(
    #     g_objective,
    #     bounds=(g_fit, 1e-5),
    #     method='bounded',
    #     options={'xatol': 1e-15}
    # )

    # Anhängen der oberen Grenze an die Liste
    g_limit_list_3D_att.append(g_limit)

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 49777.749924703196
       x: [ 1.097e+07  8.747e-16  3.820e-08]
     nit: 1
   direc: [[ 1.000e+00  0.000e+00  0.000e+00]
           [ 0.000e+00  1.000e+00  0.000e+00]
           [ 0.000e+00  0.000e+00  1.000e+00]]
    nfev: 9
49777.749924703196
 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 49776.58684088328
       x: [ 1.097e+07  8.747e-16  3.820e-08]
     nit: 1
   direc: [[ 1.000e+00  0.000e+00  0.000e+00]
           [ 0.000e+00  1.000e+00  0.000e+00]
           [ 0.000e+00  0.000e+00  1.000e+00]]
    nfev: 9
49776.58684088328
 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 49774.3467903415
       x: [ 1.097e+07  8.747e-16  3.820e-08]
     nit: 1
   direc: [[ 1.000e+00  0.000e+00  0.000e+00]
           [ 0.000e+00  1.000e+00  0.000e+00]
           [ 0.000e+00  0.000e+00  1.000e+00]]
    nfev: 9
49774.3467903415
 message: Opt

In [ ]:
# 3D-Fit in (Rydberg, Rp, g) für ein repulsives Potential

# Massenarray für den Fit
mass_array = np.array(mass_array_for_NP_fit)
# Repulsives Potential
kind = +1

# --- Ergebnis-Listen ---
g_limit_list_3D_rep = []
chi2_list_3D_rep = []
chi2_red_list_3D_rep = []
R_fit_list_3D_rep = []
Rp_fit_list_3D_rep = []


# --- Freiheitsgrade: 3 Parameter (Rydberg, Rp, g) ---
dof = len(freqs) - 3  # Anzahl Datenpunkte minus Anzahl freier Parameter

# --- Iteration über alle Massen ---
for mX in mass_array_for_NP_fit:
    # Initialwerte für Fit
    R_guess = best_Ryd
    g_guess = 1e-13
    Rp_guess = best_Rp

    # Test des chi^2-Wertes für die Startwerte der Masse mX
    print(f"Testing mX = {mX}")
    chi2_val_test = chi2_R_g_3D([R_guess,Rp_guess, g_guess], x, freqs, total_unc, mX, kind)
    print(f"Chi2 value: {chi2_val_test}")

    # Fit: gleichzeitige Optimierung von Rydberg, Rp und g mit der Powell-Methode
    res_3D_rep = minimize(
        chi2_R_g_3D,
        x0=[R_guess, Rp_guess, g_guess],
        args=(x, freqs, total_unc, mX, kind),
        bounds=[(R_guess * 0.997, R_guess * 1.003),(Rp_guess*0.997, Rp_guess*1.003), (1e-15, 1e-5)],
        method='Powell'
    )
    # print(res_3D_rep)

    # nan als Ausgabe bei dem Scheitern der Optimierung
    if not res.success:
        g_limit_list_3D_rep.append(np.nan)
        chi2_list_3D_rep.append(np.nan)
        chi2_red_list_3D_rep.append(np.nan)
        R_fit_list_3D_rep.append(np.nan)
        Rp_fit_list_3D_rep.append(np.nan)
        continue

    # Entpacken der Ergebnisse der Optimierung
    R_fit, Rp_fit, g_fit = res_3D_rep.x
    chi2_min = res_3D_rep.fun
    # print(chi2_min)
    chi2_red = chi2_min / dof

    # Speichern der Optimalen Parameter in die Listen
    R_fit_list_3D_rep.append(R_fit)
    Rp_fit_list_3D_rep.append(Rp_fit)
    chi2_list_3D_rep.append(chi2_min)
    chi2_red_list_3D_rep.append(chi2_red)

     # Gridsuche mit Interpolation für g

    # Aufspannen eines Gitters mit Stützstellen, für welches die chi^2-Werte berechnet werden
    g_grid = np.logspace(-14, -5.1, 200)
    # Berechnung der chi^2-Werte für die Stützstellen
    chi2_grid = [chi2_R_g_3D([R_fit, Rp_fit, g], x, freqs, total_unc, mX, kind) for g in g_grid]
    chi2_target = chi2_min + 1.0

    try:
        # Interpolation der chi²-Werte
        chi2_interp = interp1d(g_grid, np.array(chi2_grid) - chi2_target, kind='linear')

        # Nullstelle finden im Bereich, in dem chi² erstmals die Schwelle überschreitet mit der Nulllstellenmethode brentq
        for i in range(1, len(g_grid)):
            if (chi2_grid[i-1] - chi2_target) * (chi2_grid[i] - chi2_target) < 0:
                g_limit = brentq(chi2_interp, g_grid[i-1], g_grid[i])
                break
        # Wenn keine Nullstelle gefunden wurde, setze g_limit auf den letzten Punkt des Gitters, sodass trotzdem ein Wert zurückgegeben wird
        else:
            g_limit = g_grid[-1]  # fallback

    except (ValueError, RuntimeError):
        g_limit = g_grid[-1]  # fallback bei Interpolationsfehlern

    # Anhängen der oberen Grenze an die Liste
    g_limit_list_3D_rep.append(g_limit)

Testing mX = 1.0
Chi2 value: 18.418169995863998
 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 499795525.47744495
       x: [ 1.097e+07  8.747e-16  3.820e-06]
     nit: 1
   direc: [[ 1.000e+00  0.000e+00  0.000e+00]
           [ 0.000e+00  1.000e+00  0.000e+00]
           [ 0.000e+00  0.000e+00  1.000e+00]]
    nfev: 9
499795525.47744495
Testing mX = 1.3738237958832629
Chi2 value: 18.418169995863998
 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 499783484.4130317
       x: [ 1.097e+07  8.747e-16  3.820e-06]
     nit: 1
   direc: [[ 1.000e+00  0.000e+00  0.000e+00]
           [ 0.000e+00  1.000e+00  0.000e+00]
           [ 0.000e+00  0.000e+00  1.000e+00]]
    nfev: 9
499783484.4130317
Testing mX = 1.8873918221350972
Chi2 value: 18.418169995863998
 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 499760983.0155493
       x: [ 1.097e+07  8.747e-16  3.820e-06]
     nit: 1
   dir

In [ ]:
# Definition der Chi²-Funktion für die 2D-Optimierung in (Rydberg, g)
def chi2_R_g_2D(params, xdatas, ydata, yerr, mass, kind):
    Rydberg, coup = params
    model = freq_SM_NP_fit(xdatas, Rydberg, best_Rp, coup, mass, kind)
    return np.sum(((model - ydata) / yerr) ** 2)

# repulsives Potential
kind = +1

# Definition der Ergebnis-Listen
g_limit_list_2D_rep = []
chi2_list_2D_rep = []
chi2_red_list_2D_rep = []
R_fit_list_2D_rep = []
# Rp_fit_list = []

# Freiheitsgrade: 2 Parameter (Rydberg, g)
dof = len(freqs) - 2  # Anzahl Datenpunkte minus Anzahl freier Parameter

# Iteration über alle Massen
for mX in mass_array_for_NP_fit:
    # Initialwerte für Fit
    R_guess = best_Ryd
    g_guess = 1e-11
    # Test des chi^2-Wertes für die Startwerte der Masse mX
    print(f"Testing mX = {mX}")
    chi2_val_test = chi2_R_g_2D([R_guess, g_guess], x, freqs, total_unc, mX, kind)
    print(f"Chi2 value: {chi2_val_test}")

    # Fit: gleichzeitige Optimierung von Rydberg und g mit der Powell-Methode
    res = minimize(
        chi2_R_g_2D,
        x0=[R_guess, g_guess],
        args=(x, freqs, total_unc, mX, kind),
        bounds=[(R_guess * 0.997, R_guess * 1.003), (1e-13, 1e-7)],
        method='Powell'
    )

    # nan als Ausgabe bei dem Scheitern der Optimierung
    if not res.success:
        g_limit_list_2D_rep.append(np.nan)
        chi2_list_2D_rep.append(np.nan)
        chi2_red_list_2D_rep.append(np.nan)
        R_fit_list_2D_rep.append(np.nan)
        # Rp_fit_list.append(np.nan)
        continue
    
    # print(f"Optimization result for mX = {mX}: {res}")

    # Entpacken der Ergebnisse der Optimierung
    R_fit, g_fit = res.x
    chi2_min = res.fun
    print("minimales chi2", chi2_min)
    chi2_red = chi2_min / dof

    # Speichern der optimalen Parameter in die Listen
    R_fit_list_2D_rep.append(R_fit)
    chi2_list_2D_rep.append(chi2_min)
    chi2_red_list_2D_rep.append(chi2_red)

    # Gridsuche mit Interpolation für g
    # Aufspannen eines Gitters mit Stützstellen, für welches die chi^2-Werte berechnet werden
    g_grid = np.logspace(-13, -7, 200)
    # Berechnung der chi^2-Werte für die Stützstellen
    chi2_grid = [chi2_R_g_2D([R_fit, g], x, freqs, total_unc, mX, kind) for g in g_grid]
    # grenzen der 1σ-Schwelle
    chi2_target = chi2_min + 1.0
    # print(f"Chi2 grid for mX = {mX}: {chi2_grid}")

    # Interpolation der chi²-Werte und Bestimmung der Nullstelle mit brentq
    try:
        chi2_interp = interp1d(g_grid, np.array(chi2_grid) - chi2_target, kind='linear')

        # Find the maximum g within the 1σ interval
        for i in range(1, len(g_grid)):
            if (chi2_grid[i - 1] - chi2_target) * (chi2_grid[i] - chi2_target) < 0:
                g_limit = brentq(chi2_interp, g_grid[i - 1], g_grid[i])
                break
        # else:
        #     g_limit = g_grid[-1]  # fallback

    except (ValueError, RuntimeError):
        g_limit = g_grid[-1]  # fallback in case of interpolation errors

    # Speichern der oberen Grenze in die Liste
    g_limit_list_2D_rep.append(g_limit)



Testing mX = 1.0
Chi2 value: 18.42113147156086
Optimization result for mX = 1.0:  message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 49972.37286400976
       x: [ 1.097e+07  3.820e-08]
     nit: 1
   direc: [[ 1.000e+00  0.000e+00]
           [ 0.000e+00  1.000e+00]]
    nfev: 8
minimales chi2 49972.37286400976
Testing mX = 1.3738237958832629
Chi2 value: 18.42121651454339
Optimization result for mX = 1.3738237958832629:  message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 49971.1783438664
       x: [ 1.097e+07  3.820e-08]
     nit: 1
   direc: [[ 1.000e+00  0.000e+00]
           [ 0.000e+00  1.000e+00]]
    nfev: 8
minimales chi2 49971.1783438664
Testing mX = 1.8873918221350972
Chi2 value: 18.42112848952626
Optimization result for mX = 1.8873918221350972:  message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 49968.92166635093
       x: [ 1.097e+07  3.820e-08]
     nit: 1
   direc: [[ 1.000e+

In [ ]:
# 2D-Optimierung in (Rydberg, g) für ein attraktives Potential

kind = -1

# Definition der Ergebnis-Listen
g_limit_list_2D_att = []
chi2_list_2D_att = []
chi2_red_list_2D_att = []
R_fit_list_2D_att = []

# Freiheitsgrade: 2 Parameter (Rydberg, g)
dof = len(freqs) - 2  # Number of data points minus number of free parameters

# iteration über alle Massen
for mX in mass_array_for_NP_fit:
    # Initialwerte für Fit
    R_guess = best_Ryd
    g_guess = 1e-10

    # Test des chi^2-Wertes für die Startwerte der Masse mX
    print(f"Testing mX = {mX}")
    chi2_val_test = chi2_R_g_2D([best_Ryd, 1e-13], x, freqs, total_unc, mX, kind)
    print(f"Chi2 value: {chi2_val_test}")

    # 2D-Optimierung der Chi²-Funktion mit dem Powell-Algorithmus
    res = minimize(
        chi2_R_g_2D,
        x0=[R_guess, g_guess],
        args=(x, freqs, total_unc, mX, kind),
        bounds=[(R_guess * 0.997, R_guess * 1.003), (1e-13, 1e-7)],
        method='Powell'
        # method = "trust-constr"
        # method = "BFGS"
    )

    # nan als Ausgabe bei dem Scheitern der Optimierung
    if not res.success:
        g_limit_list_2D_att.append(np.nan)
        chi2_list_2D_att.append(np.nan)
        chi2_red_list_2D_att.append(np.nan)
        R_fit_list_2D_att.append(np.nan)
        # Rp_fit_list.append(np.nan)
        continue
    
    # # Test der Ausgaben
    # print(f"Optimization result for mX = {mX}: {res.x}")

    # Ausgabe der Anpassungsprarameter
    R_fit, g_fit = res.x
    chi2_min = res.fun
    chi2_red = chi2_min / dof

    # Speichern der Optimalen Parameter
    R_fit_list_2D_att.append(R_fit)
    chi2_list_2D_att.append(chi2_min)
    chi2_red_list_2D_att.append(chi2_red)

    # Gridsuche mit Interpolation für g
    # Aufspannen eines Gitters mit Stützstellen, für welches die chi^2-Werte berechnet werden
    g_grid = np.logspace(-13, -7, 200)
    # Berechnung der chi^2-Werte für die Stützstellen
    chi2_grid = [chi2_R_g_2D([R_fit, g], x, freqs, total_unc, mX, kind) for g in g_grid]
    # grenzen der 1σ-Schwelle
    chi2_target = chi2_min + 1.0
    # print(f"Chi2 grid for mX = {mX}: {chi2_grid}")

    # Interpolation der chi²-Werte und Bestimmung der Nullstelle mit brentq
    try:
        chi2_interp = interp1d(g_grid, np.array(chi2_grid) - chi2_target, kind='linear')

        # Find the maximum g within the 1σ interval
        for i in range(1, len(g_grid)):
            if (chi2_grid[i - 1] - chi2_target) * (chi2_grid[i] - chi2_target) < 0:
                g_limit = brentq(chi2_interp, g_grid[i - 1], g_grid[i])
                break
        # else:
        #     g_limit = g_grid[-1]  # fallback

    except (ValueError, RuntimeError):
        g_limit = g_grid[-1]  # fallback in case of interpolation errors

    # Speichern der oberen Grenze
    g_limit_list_2D_att.append(g_limit)

Testing mX = 1.0
Chi2 value: 18.418154175300106
Chi2 grid for mX = 1.0: [np.float64(18.760012777203894), np.float64(18.760086214664785), np.float64(18.760042252798176), np.float64(18.759909969936587), np.float64(18.75986824520031), np.float64(18.75979835524431), np.float64(18.759656270262816), np.float64(18.75962325296856), np.float64(18.75957036134827), np.float64(18.759490949174307), np.float64(18.759408797165158), np.float64(18.75924733743532), np.float64(18.7592173691874), np.float64(18.75916303337093), np.float64(18.75896229789484), np.float64(18.758708454136382), np.float64(18.75860627651886), np.float64(18.75844348206811), np.float64(18.758355518410408), np.float64(18.75815516674538), np.float64(18.75812634715043), np.float64(18.757732472765984), np.float64(18.757598567621525), np.float64(18.757374020361596), np.float64(18.75717790509694), np.float64(18.756954304874778), np.float64(18.756539791603842), np.float64(18.756330951961004), np.float64(18.756016171855688), np.float64(18